### Importa as bibliotecas

In [37]:
import pandas as pd
import requests
import json

### Funções

In [38]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [39]:
def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

### Extract

In [40]:
df = pd.read_csv('assets/SDW2023.csv')
user_ids = df['UserID'].tolist()
user_ids

[4732, 4734, 4735]

In [41]:
users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 4732,
    "name": "Raimunda",
    "account": {
      "id": 5013,
      "number": "00123-4",
      "agency": "0555",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4595,
      "number": "1452 2578 4521 1000",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 4734,
    "name": "Renata Ingrata",
    "account": {
      "id": 5015,
      "number": "00125-4",
      "agency": "0555",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4597,
      "number": "1452 2578 4521 10010",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 4735,
    "name": "Jo\u00e3o do Caminh\u00e3o",
    "account": {
      "id": 5016,
      "number": "00126-4",
      "agency": "0555",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 4598,
      "number": "1452 2578 4521 10016",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]
